In [ ]:
from phe import paillier
import numpy as np
from sklearn.datasets import make_classification
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score


from sklearn.model_selection import train_test_split

class Client:
    def __init__(self, id, public_key, private_key, data, labels):
        self.id = id
        self.public_key = public_key
        self.private_key = private_key

        # Split the data into train and test sets
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(data, labels, test_size=0.3, random_state=42)

        self.model_weights = None
    
    def train_local_model(self):
        clf = SGDClassifier(loss='log_loss', max_iter=1000, learning_rate='constant', eta0=0.01)
        clf.fit(self.X_train, self.y_train)
        self.model_weights = clf.coef_[0]
        return self.model_weights

    def encrypt_weights(self):
        return [self.public_key.encrypt(x) for x in self.model_weights]
    
    def decrypt_weights(self, encrypted_weights):
        return np.array([self.private_key.decrypt(x) for x in encrypted_weights])
    


class Server:
    def __init__(self, public_key):
        self.public_key = public_key
        self.encrypted_models = []

    def receive_encrypted_model(self, encrypted_model):
        self.encrypted_models.append(encrypted_model)

    def aggregate_encrypted_models(self, num_clients):
        n_weights = len(self.encrypted_models[0])
        aggregated_model = []
        for i in range(n_weights):
            sum_enc = sum(client_weights[i] for client_weights in self.encrypted_models)
            avg_enc = sum_enc * (1 / num_clients)
            aggregated_model.append(avg_enc)
        return aggregated_model
    

def federated_training(num_clients=3, num_rounds=3):
    # Step 1: Create data
    X, y = make_classification(n_samples=300, n_features=5, n_informative=3, n_classes=2)
    split_data = np.array_split(X, num_clients)
    split_labels = np.array_split(y, num_clients)

    # Step 2: Generate key pair
    public_key, private_key = paillier.generate_paillier_keypair()

    # Step 3: Initialize clients
    clients = []
    for i in range(num_clients):
        clients.append(Client(i, public_key, private_key, split_data[i], split_labels[i]))

    # Step 4: Initialize server
    server = Server(public_key)

    for round in range(num_rounds):
        print(f"\n🔁 Round {round + 1} - Local training and encryption")

        server.encrypted_models.clear()
        for client in clients:
            local_weights = client.train_local_model()
            enc_weights = client.encrypt_weights()
            server.receive_encrypted_model(enc_weights)
            print(f"Client {client.id} encrypted and sent weights.")

        print("🔐 Server aggregating encrypted models...")
        encrypted_global_model = server.aggregate_encrypted_models(num_clients)

        for client in clients:
            decrypted_global_model = client.decrypt_weights(encrypted_global_model)
            client.model_weights = decrypted_global_model  # Update local model weights
            print(f"Client {client.id} decrypted global model.")

    return clients

# ------------------------------
# Evaluation
# ------------------------------

def evaluate_global_model(clients):
    for client in clients:
        clf = SGDClassifier()
        clf.coef_ = client.model_weights.reshape(1, -1)
        clf.intercept_ = np.array([0])
        clf.classes_ = np.array([0, 1])  # Manually set class labels

        preds = clf.predict(client.X_test)
        acc = accuracy_score(client.y_test, preds)
        print(f"Client {client.id} Test Accuracy: {acc:.2f}")


# ------------------------------
# Run the Whole Pipeline
# ------------------------------

clients = federated_training()
evaluate_global_model(clients)

ankara messi


In [3]:
from phe import paillier
import numpy as np
from sklearn.datasets import make_classification
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import random


class Node:
    def __init__(self, id, data, labels):
        self.id = id
        self.public_key, self.private_key = paillier.generate_paillier_keypair()
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(data, labels, test_size=0.3, random_state=42)
        self.model_weights = None
        self.intercept = 0

    def train_model(self):
        clf = SGDClassifier(loss='log_loss', max_iter=1000, learning_rate='constant', eta0=0.01)
        if self.model_weights is not None:
            clf.coef_ = self.model_weights.reshape(1, -1)
            clf.intercept_ = np.array([self.intercept])
            clf.classes_ = np.array([0, 1])
        clf.fit(self.X_train, self.y_train)
        self.model_weights = clf.coef_[0]
        self.intercept = clf.intercept_[0]

    def encrypt_weights_for(self, recipient_public_key):
        encrypted_weights = [recipient_public_key.encrypt(x) for x in self.model_weights]
        encrypted_intercept = recipient_public_key.encrypt(self.intercept)
        return encrypted_weights, encrypted_intercept

    def decrypt_weights(self, encrypted_weights, encrypted_intercept):
        self.model_weights = np.array([self.private_key.decrypt(x) for x in encrypted_weights])
        self.intercept = self.private_key.decrypt(encrypted_intercept)

    def evaluate(self):
        clf = SGDClassifier()
        clf.coef_ = self.model_weights.reshape(1, -1)
        clf.intercept_ = np.array([self.intercept])
        clf.classes_ = np.array([0, 1])
        preds = clf.predict(self.X_test)
        return accuracy_score(self.y_test, preds)


def secure_gossip_training(num_nodes=3, num_rounds=3):
    X, y = make_classification(n_samples=300, n_features=5, n_informative=3, n_classes=2)
    split_data = np.array_split(X, num_nodes)
    split_labels = np.array_split(y, num_nodes)

    # Create nodes
    nodes = [Node(i, split_data[i], split_labels[i]) for i in range(num_nodes)]

    # Start with one node training initial model
    print(f"🎯 Initial Training at Node 0")
    nodes[0].train_model()

    # Gossip Rounds
    current_node_idx = 0
    for rnd in range(num_rounds):
        sender = nodes[current_node_idx]
        receiver_idx = (current_node_idx + 1) % num_nodes
        receiver = nodes[receiver_idx]

        print(f"\n🔁 Round {rnd+1}: Node {sender.id} → Node {receiver.id}")

        # Sender encrypts model for receiver
        enc_weights, enc_intercept = sender.encrypt_weights_for(receiver.public_key)
        print(f"Node {sender.id} encrypted model for Node {receiver.id}")

        # Receiver decrypts and trains
        receiver.decrypt_weights(enc_weights, enc_intercept)
        print(f"Node {receiver.id} decrypted model")

        receiver.train_model()
        print(f"Node {receiver.id} trained model")

        # Move to next node
        current_node_idx = receiver_idx

    return nodes


def evaluate_nodes(nodes):
    print("\n📊 Evaluation Results:")
    for node in nodes:
        acc = node.evaluate()
        print(f"Node {node.id} Test Accuracy: {acc:.2f}")


# ------------------------------
# Run Secure Gossip Learning
# ------------------------------
nodes = secure_gossip_training()
evaluate_nodes(nodes)


🎯 Initial Training at Node 0

🔁 Round 1: Node 0 → Node 1
Node 0 encrypted model for Node 1
Node 1 decrypted model
Node 1 trained model

🔁 Round 2: Node 1 → Node 2
Node 1 encrypted model for Node 2
Node 2 decrypted model
Node 2 trained model

🔁 Round 3: Node 2 → Node 0
Node 2 encrypted model for Node 0
Node 0 decrypted model
Node 0 trained model

📊 Evaluation Results:
Node 0 Test Accuracy: 0.87
Node 1 Test Accuracy: 0.83
Node 2 Test Accuracy: 0.87


In [6]:
import numpy as np
import random
from sklearn.datasets import make_classification
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# Large prime and primitive root for Diffie-Hellman
q = 7919  # A small prime for demonstration (use a large one in practice)
alpha = 2  # Primitive root modulo q

def modular_pow(base, exponent, modulus):
    result = 1
    base = base % modulus
    while exponent > 0:
        if exponent % 2:
            result = (result * base) % modulus
        exponent = exponent >> 1
        base = (base * base) % modulus
    return result

class Node:
    def __init__(self, id, data, labels):
        self.id = id
        self.private_key = random.randint(2, q - 2)
        self.public_key = modular_pow(alpha, self.private_key, q)

        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
            data, labels, test_size=0.3, random_state=42)

        self.model_weights = None
        self.intercept = 0

    def compute_shared_key(self, other_public_key):
        return modular_pow(other_public_key, self.private_key, q)

    def encrypt_model(self, weights, intercept, shared_key):
        enc_weights = [w + shared_key for w in weights]
        enc_intercept = intercept + shared_key
        return enc_weights, enc_intercept

    def decrypt_model(self, enc_weights, enc_intercept, shared_key):
        self.model_weights = np.array([w - shared_key for w in enc_weights])
        self.intercept = enc_intercept - shared_key

    def train_model(self):
        clf = SGDClassifier(loss='log_loss', max_iter=1000, learning_rate='constant', eta0=0.01)
        if self.model_weights is not None:
            clf.coef_ = self.model_weights.reshape(1, -1)
            clf.intercept_ = np.array([self.intercept])
            clf.classes_ = np.array([0, 1])
        clf.fit(self.X_train, self.y_train)
        self.model_weights = clf.coef_[0]
        self.intercept = clf.intercept_[0]

    def evaluate(self):
        clf = SGDClassifier()
        clf.coef_ = self.model_weights.reshape(1, -1)
        clf.intercept_ = np.array([self.intercept])
        clf.classes_ = np.array([0, 1])
        preds = clf.predict(self.X_test)
        return accuracy_score(self.y_test, preds)

def secure_gossip_training_dh(num_nodes=3, num_rounds=3):
    X, y = make_classification(n_samples=300, n_features=5, n_informative=3, n_classes=2)
    split_data = np.array_split(X, num_nodes)
    split_labels = np.array_split(y, num_nodes)

    nodes = [Node(i, split_data[i], split_labels[i]) for i in range(num_nodes)]

    print(f"\n🎯 Initial Training at Node 0")
    nodes[0].train_model()

    current_node_idx = 0
    for rnd in range(num_rounds):
        print(f"\n🔁 Round {rnd + 1}")
        for _ in range(num_nodes):
            sender = nodes[current_node_idx]
            receiver_idx = (current_node_idx + 1) % num_nodes
            receiver = nodes[receiver_idx]

            shared_key = sender.compute_shared_key(receiver.public_key)
            enc_weights, enc_intercept = sender.encrypt_model(sender.model_weights, sender.intercept, shared_key)
            print(f"Node {sender.id} encrypted model for Node {receiver.id} with shared key")

            shared_key_receiver = receiver.compute_shared_key(sender.public_key)
            receiver.decrypt_model(enc_weights, enc_intercept, shared_key_receiver)
            print(f"Node {receiver.id} decrypted model")

            receiver.train_model()
            print(f"Node {receiver.id} trained model")

            current_node_idx = receiver_idx

    return nodes

def evaluate_nodes(nodes):
    print("\n📊 Evaluation Results:")
    for node in nodes:
        acc = node.evaluate()
        print(f"Node {node.id} Test Accuracy: {acc:.2f}")

# Run
nodes = secure_gossip_training_dh()
evaluate_nodes(nodes)



🎯 Initial Training at Node 0

🔁 Round 1
Node 0 encrypted model for Node 1 with shared key
Node 1 decrypted model
Node 1 trained model
Node 1 encrypted model for Node 2 with shared key
Node 2 decrypted model
Node 2 trained model
Node 2 encrypted model for Node 0 with shared key
Node 0 decrypted model
Node 0 trained model

🔁 Round 2
Node 0 encrypted model for Node 1 with shared key
Node 1 decrypted model
Node 1 trained model
Node 1 encrypted model for Node 2 with shared key
Node 2 decrypted model
Node 2 trained model
Node 2 encrypted model for Node 0 with shared key
Node 0 decrypted model
Node 0 trained model

🔁 Round 3
Node 0 encrypted model for Node 1 with shared key
Node 1 decrypted model
Node 1 trained model
Node 1 encrypted model for Node 2 with shared key
Node 2 decrypted model
Node 2 trained model
Node 2 encrypted model for Node 0 with shared key
Node 0 decrypted model
Node 0 trained model

📊 Evaluation Results:
Node 0 Test Accuracy: 0.83
Node 1 Test Accuracy: 0.77
Node 2 Test Ac

In [8]:
import numpy as np
import random
from sklearn.datasets import make_classification
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# Set global seed for reproducibility
random.seed(42)
np.random.seed(42)

# Large prime and primitive root for Diffie-Hellman
q = 7919  # A small prime for demonstration (use a large one in practice)
alpha = 2  # Primitive root modulo q

def modular_pow(base, exponent, modulus):
    result = 1
    base = base % modulus
    while exponent > 0:
        if exponent % 2:
            result = (result * base) % modulus
        exponent = exponent >> 1
        base = (base * base) % modulus
    return result

class Node:
    def __init__(self, id, data, labels):
        self.id = id
        self.private_key = random.randint(2, q - 2)
        self.public_key = modular_pow(alpha, self.private_key, q)

        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
            data, labels, test_size=0.3, random_state=42)

        self.model_weights = None
        self.intercept = 0

    def compute_shared_key(self, other_public_key):
        return modular_pow(other_public_key, self.private_key, q)

    def encrypt_model(self, weights, intercept, shared_key):
        enc_weights = [w + shared_key for w in weights]
        enc_intercept = intercept + shared_key
        return enc_weights, enc_intercept

    def decrypt_model(self, enc_weights, enc_intercept, shared_key):
        self.model_weights = np.array([w - shared_key for w in enc_weights])
        self.intercept = enc_intercept - shared_key

    def train_model(self):
        clf = SGDClassifier(loss='log_loss', max_iter=1000, learning_rate='constant', eta0=0.01, random_state=42)
        if self.model_weights is not None:
            clf.coef_ = self.model_weights.reshape(1, -1)
            clf.intercept_ = np.array([self.intercept])
            clf.classes_ = np.array([0, 1])
        clf.fit(self.X_train, self.y_train)
        self.model_weights = clf.coef_[0]
        self.intercept = clf.intercept_[0]

    def evaluate(self):
        clf = SGDClassifier(loss='log_loss', max_iter=1000, learning_rate='constant', eta0=0.01, random_state=42)
        clf.coef_ = self.model_weights.reshape(1, -1)
        clf.intercept_ = np.array([self.intercept])
        clf.classes_ = np.array([0, 1])
        preds = clf.predict(self.X_test)
        return accuracy_score(self.y_test, preds)

def secure_gossip_training_dh(num_nodes=3, num_rounds=3):
    X, y = make_classification(n_samples=300, n_features=5, n_informative=3, n_classes=2, random_state=42)
    split_data = np.array_split(X, num_nodes)
    split_labels = np.array_split(y, num_nodes)

    nodes = [Node(i, split_data[i], split_labels[i]) for i in range(num_nodes)]

    print(f"\n🎯 Initial Training at Node 0")
    nodes[0].train_model()

    for rnd in range(num_rounds):
        print(f"\n🔁 Round {rnd + 1}")
        for i in range(num_nodes):
            sender = nodes[i]
            receiver = nodes[(i + 1) % num_nodes]

            shared_key = sender.compute_shared_key(receiver.public_key)
            enc_weights, enc_intercept = sender.encrypt_model(sender.model_weights, sender.intercept, shared_key)
            print(f"Node {sender.id} encrypted model for Node {receiver.id} with shared key")

            shared_key_receiver = receiver.compute_shared_key(sender.public_key)
            receiver.decrypt_model(enc_weights, enc_intercept, shared_key_receiver)
            print(f"Node {receiver.id} decrypted model")

            receiver.train_model()
            print(f"Node {receiver.id} trained model")

        print("\n📊 Accuracy after Round", rnd + 1)
        for node in nodes:
            acc = node.evaluate()
            print(f"Node {node.id} Test Accuracy: {acc:.2f}")

    return nodes

def evaluate_nodes(nodes):
    print("\n📊 Final Evaluation Results:")
    for node in nodes:
        acc = node.evaluate()
        print(f"Node {node.id} Test Accuracy: {acc:.2f}")

# Run
nodes = secure_gossip_training_dh()
evaluate_nodes(nodes)



🎯 Initial Training at Node 0

🔁 Round 1
Node 0 encrypted model for Node 1 with shared key
Node 1 decrypted model
Node 1 trained model
Node 1 encrypted model for Node 2 with shared key
Node 2 decrypted model
Node 2 trained model
Node 2 encrypted model for Node 0 with shared key
Node 0 decrypted model
Node 0 trained model

📊 Accuracy after Round 1
Node 0 Test Accuracy: 0.77
Node 1 Test Accuracy: 0.93
Node 2 Test Accuracy: 0.93

🔁 Round 2
Node 0 encrypted model for Node 1 with shared key
Node 1 decrypted model
Node 1 trained model
Node 1 encrypted model for Node 2 with shared key
Node 2 decrypted model
Node 2 trained model
Node 2 encrypted model for Node 0 with shared key
Node 0 decrypted model
Node 0 trained model

📊 Accuracy after Round 2
Node 0 Test Accuracy: 0.77
Node 1 Test Accuracy: 0.93
Node 2 Test Accuracy: 0.93

🔁 Round 3
Node 0 encrypted model for Node 1 with shared key
Node 1 decrypted model
Node 1 trained model
Node 1 encrypted model for Node 2 with shared key
Node 2 decrypte

In [13]:
import numpy as np
import random
from sklearn.datasets import make_classification
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split


# Large prime and primitive root for Diffie-Hellman
q = 7919  # A small prime for demonstration (use a large one in practice)
alpha = 2  # Primitive root modulo q

def modular_pow(base, exponent, modulus):
    result = 1
    base = base % modulus
    while exponent > 0:
        if exponent % 2:
            result = (result * base) % modulus
        exponent = exponent >> 1
        base = (base * base) % modulus
    return result

class Node:
    def __init__(self, id, data, labels):
        self.id = id
        self.private_key = random.randint(2, q - 2)
        self.public_key = modular_pow(alpha, self.private_key, q)

        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
            data, labels, test_size=0.3, random_state=42)

        self.model_weights = None
        self.intercept = 0

    def compute_shared_key(self, other_public_key):
        return modular_pow(other_public_key, self.private_key, q)

    def encrypt_model(self, weights, intercept, shared_key):
        enc_weights = [w + shared_key for w in weights]
        enc_intercept = intercept + shared_key
        return enc_weights, enc_intercept

    def decrypt_model(self, enc_weights, enc_intercept, shared_key):
        self.model_weights = np.array([w - shared_key for w in enc_weights])
        self.intercept = enc_intercept - shared_key

    def train_model(self):
        clf = SGDClassifier(loss='log_loss', max_iter=1000, learning_rate='constant', eta0=0.01, random_state=42)
        if self.model_weights is not None:
            clf.coef_ = self.model_weights.reshape(1, -1)
            clf.intercept_ = np.array([self.intercept])
            clf.classes_ = np.array([0, 1])
        clf.fit(self.X_train, self.y_train)
        self.model_weights = clf.coef_[0]
        self.intercept = clf.intercept_[0]

    def evaluate(self):
        clf = SGDClassifier()
        clf.coef_ = self.model_weights.reshape(1, -1)
        clf.intercept_ = np.array([self.intercept])
        clf.classes_ = np.array([0, 1])
        preds = clf.predict(self.X_test)
        return accuracy_score(self.y_test, preds)

def secure_gossip_training_dh(num_nodes=3, num_rounds=3):
    X, y = make_classification(n_samples=300, n_features=5, n_informative=3, n_classes=2, random_state=42)
    split_data = np.array_split(X, num_nodes)
    split_labels = np.array_split(y, num_nodes)

    nodes = [Node(i, split_data[i], split_labels[i]) for i in range(num_nodes)]

    print(f"\n🎯 Initial Training at Node 0")
    nodes[0].train_model()

    current_holder = 0
    total_transfers = num_rounds * num_nodes

    for t in range(total_transfers):
        sender = nodes[current_holder]
        receiver = nodes[(current_holder + 1) % num_nodes]

        shared_key = sender.compute_shared_key(receiver.public_key)
        enc_weights, enc_intercept = sender.encrypt_model(sender.model_weights, sender.intercept, shared_key)
        print(f"Node {sender.id} encrypted model for Node {receiver.id} with shared key")

        shared_key_recv = receiver.compute_shared_key(sender.public_key)
        receiver.decrypt_model(enc_weights, enc_intercept, shared_key_recv)
        print(f"Node {receiver.id} decrypted model")

        receiver.train_model()
        print(f"Node {receiver.id} trained model")

        current_holder = (current_holder + 1) % num_nodes

        if (t + 1) % num_nodes == 0:
            print(f"\n📊 Accuracy after Round {(t + 1) // num_nodes}")
            for node in nodes:
                acc = node.evaluate()
                print(f"Node {node.id} Test Accuracy: {acc:.2f}")

    return nodes

def evaluate_nodes(nodes):
    print("\n📊 Final Evaluation Results:")
    for node in nodes:
        acc = node.evaluate()
        print(f"Node {node.id} Test Accuracy: {acc:.2f}")

# Run
nodes = secure_gossip_training_dh()
evaluate_nodes(nodes)



🎯 Initial Training at Node 0
Node 0 encrypted model for Node 1 with shared key
Node 1 decrypted model
Node 1 trained model
Node 1 encrypted model for Node 2 with shared key
Node 2 decrypted model
Node 2 trained model
Node 2 encrypted model for Node 0 with shared key
Node 0 decrypted model
Node 0 trained model

📊 Accuracy after Round 1
Node 0 Test Accuracy: 0.77
Node 1 Test Accuracy: 0.93
Node 2 Test Accuracy: 0.93
Node 0 encrypted model for Node 1 with shared key
Node 1 decrypted model
Node 1 trained model
Node 1 encrypted model for Node 2 with shared key
Node 2 decrypted model
Node 2 trained model
Node 2 encrypted model for Node 0 with shared key
Node 0 decrypted model
Node 0 trained model

📊 Accuracy after Round 2
Node 0 Test Accuracy: 0.77
Node 1 Test Accuracy: 0.93
Node 2 Test Accuracy: 0.93
Node 0 encrypted model for Node 1 with shared key
Node 1 decrypted model
Node 1 trained model
Node 1 encrypted model for Node 2 with shared key
Node 2 decrypted model
Node 2 trained model
Node

In [18]:
import numpy as np
import random
from sklearn.datasets import make_classification
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Diffie-Hellman parameters
q = 7919  # Small prime for demo; use large prime in real systems
alpha = 2  # Primitive root modulo q

def modular_pow(base, exponent, modulus):
    result = 1
    base = base % modulus
    while exponent > 0:
        if exponent % 2:
            result = (result * base) % modulus
        exponent = exponent >> 1
        base = (base * base) % modulus
    return result

class Node:
    def __init__(self, id, data, labels):
        self.id = id
        self.private_key = random.randint(2, q - 2)
        self.public_key = modular_pow(alpha, self.private_key, q)
        print(data.shape)
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
            data, labels, test_size=0.2, random_state=42)

        self.model_weights = None
        self.intercept = 0

    def compute_shared_key(self, other_public_key):
        return modular_pow(other_public_key, self.private_key, q)

    def encrypt_model(self, weights, intercept, shared_key):
        enc_weights = [w + shared_key for w in weights]
        enc_intercept = intercept + shared_key
        return enc_weights, enc_intercept

    def decrypt_model(self, enc_weights, enc_intercept, shared_key):
        self.model_weights = np.array([w - shared_key for w in enc_weights])
        self.intercept = enc_intercept - shared_key

    def train_model(self):
        clf = SGDClassifier(loss='log_loss', max_iter=1000, learning_rate='constant', eta0=0.01, random_state=42)
        if self.model_weights is not None:
            clf.coef_ = self.model_weights.reshape(1, -1)
            clf.intercept_ = np.array([self.intercept])
            clf.classes_ = np.array([0, 1])
        clf.fit(self.X_train, self.y_train)
        self.model_weights = clf.coef_[0]
        self.intercept = clf.intercept_[0]

    def evaluate(self):
        clf = SGDClassifier()
        clf.coef_ = self.model_weights.reshape(1, -1)
        clf.intercept_ = np.array([self.intercept])
        clf.classes_ = np.array([0, 1])
        preds = clf.predict(self.X_test)
        return accuracy_score(self.y_test, preds)

def secure_gossip_training_dh(num_nodes=3, num_rounds=3):
    # Load and preprocess dataset
    data = load_breast_cancer()
    X, y = data.data, data.target

    print(X.shape)
    # Normalize features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Split data across nodes
    split_data = np.array_split(X_scaled, num_nodes)
    split_labels = np.array_split(y, num_nodes)

    # Initialize nodes
    nodes = [Node(i, split_data[i], split_labels[i]) for i in range(num_nodes)]

    print(f"\n🎯 Initial Training at Node 0")
    nodes[0].train_model()

    current_holder = 0
    total_transfers = num_rounds * num_nodes

    for t in range(total_transfers):
        sender = nodes[current_holder]
        receiver = nodes[(current_holder + 1) % num_nodes]

        shared_key = sender.compute_shared_key(receiver.public_key)
        enc_weights, enc_intercept = sender.encrypt_model(sender.model_weights, sender.intercept, shared_key)
        print(f"Node {sender.id} encrypted model for Node {receiver.id} with shared key")

        shared_key_recv = receiver.compute_shared_key(sender.public_key)
        receiver.decrypt_model(enc_weights, enc_intercept, shared_key_recv)
        print(f"Node {receiver.id} decrypted model")

        receiver.train_model()
        print(f"Node {receiver.id} trained model")

        current_holder = (current_holder + 1) % num_nodes

        if (t + 1) % num_nodes == 0:
            print(f"\n📊 Accuracy after Round {(t + 1) // num_nodes}")
            for node in nodes:
                acc = node.evaluate()
                print(f"Node {node.id} Test Accuracy: {acc}")

    return nodes

def evaluate_nodes(nodes):
    print("\n📊 Final Evaluation Results:")
    for node in nodes:
        acc = node.evaluate()
        print(f"Node {node.id} Test Accuracy: {acc}")

# Run
nodes = secure_gossip_training_dh()
evaluate_nodes(nodes)


(569, 30)
(190, 30)
(190, 30)
(189, 30)

🎯 Initial Training at Node 0
Node 0 encrypted model for Node 1 with shared key
Node 1 decrypted model
Node 1 trained model
Node 1 encrypted model for Node 2 with shared key
Node 2 decrypted model
Node 2 trained model
Node 2 encrypted model for Node 0 with shared key
Node 0 decrypted model
Node 0 trained model

📊 Accuracy after Round 1
Node 0 Test Accuracy: 1.0
Node 1 Test Accuracy: 0.9736842105263158
Node 2 Test Accuracy: 0.9210526315789473
Node 0 encrypted model for Node 1 with shared key
Node 1 decrypted model
Node 1 trained model
Node 1 encrypted model for Node 2 with shared key
Node 2 decrypted model
Node 2 trained model
Node 2 encrypted model for Node 0 with shared key
Node 0 decrypted model
Node 0 trained model

📊 Accuracy after Round 2
Node 0 Test Accuracy: 1.0
Node 1 Test Accuracy: 0.9736842105263158
Node 2 Test Accuracy: 0.9210526315789473
Node 0 encrypted model for Node 1 with shared key
Node 1 decrypted model
Node 1 trained model
Node

In [27]:
import numpy as np
import random
from sklearn.datasets import make_classification
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Diffie-Hellman parameters
q = 7919  # Small prime for demo; use large prime in real systems
alpha = 2  # Primitive root modulo q

def modular_pow(base, exponent, modulus):
    result = 1
    base = base % modulus
    while exponent > 0:
        if exponent % 2:
            result = (result * base) % modulus
        exponent = exponent >> 1
        base = (base * base) % modulus
    return result

class Node:
    def __init__(self, id, data, labels):
        self.id = id
        self.private_key = random.randint(2, q - 2)
        self.public_key = modular_pow(alpha, self.private_key, q)
        print(data.shape)
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
            data, labels, test_size=0.2, random_state=42)

        self.model_weights = None
        self.intercept = 0

    def compute_shared_key(self, other_public_key):
        return modular_pow(other_public_key, self.private_key, q)

    def encrypt_model(self, weights, intercept, shared_key):
        enc_weights = [w + shared_key for w in weights]
        enc_intercept = intercept + shared_key
        return enc_weights, enc_intercept

    def decrypt_model(self, enc_weights, enc_intercept, shared_key):
        self.model_weights = np.array([w - shared_key for w in enc_weights])
        self.intercept = enc_intercept - shared_key

    def train_model(self):
        clf = SGDClassifier(loss='log_loss', max_iter=1000, learning_rate='constant', eta0=0.01, random_state=42)
        if self.model_weights is not None:
            clf.coef_ = self.model_weights.reshape(1, -1)
            clf.intercept_ = np.array([self.intercept])
            clf.classes_ = np.array([0, 1])
        clf.partial_fit(self.X_train, self.y_train,classes=[0,1])
        self.model_weights = clf.coef_[0]
        self.intercept = clf.intercept_[0]

    def evaluate(self):
        clf = SGDClassifier()
        clf.coef_ = self.model_weights.reshape(1, -1)
        clf.intercept_ = np.array([self.intercept])
        clf.classes_ = np.array([0, 1])
        preds = clf.predict(self.X_test)
        return accuracy_score(self.y_test, preds)

def secure_gossip_training_dh(num_nodes=3, num_rounds=3):
    # Load and preprocess dataset
    X, y = make_classification(n_samples=1000, n_features=30, n_classes=2,
                           n_informative=15, n_redundant=5, flip_y=0.1,
                           random_state=42)


    print(X.shape)
    # Normalize features
    

    # Split data across nodes
    split_data = np.array_split(X, num_nodes)
    split_labels = np.array_split(y, num_nodes)

    # Initialize nodes
    nodes = [Node(i, split_data[i], split_labels[i]) for i in range(num_nodes)]

    print(f"\n🎯 Initial Training at Node 0")
    nodes[0].train_model()

    current_holder = 0
    total_transfers = num_rounds * num_nodes

    for t in range(total_transfers):
        sender = nodes[current_holder]
        receiver = nodes[(current_holder + 1) % num_nodes]

        shared_key = sender.compute_shared_key(receiver.public_key)
        enc_weights, enc_intercept = sender.encrypt_model(sender.model_weights, sender.intercept, shared_key)
        print(f"Node {sender.id} encrypted model for Node {receiver.id} with shared key")

        shared_key_recv = receiver.compute_shared_key(sender.public_key)
        receiver.decrypt_model(enc_weights, enc_intercept, shared_key_recv)
        print(f"Node {receiver.id} decrypted model")

        receiver.train_model()
        print(f"Node {receiver.id} trained model")

        current_holder = (current_holder + 1) % num_nodes

        if (t + 1) % num_nodes == 0:
            print(f"\n📊 Accuracy after Round {(t + 1) // num_nodes}")
            for node in nodes:
                acc = node.evaluate()
                print(f"Node {node.id} Test Accuracy: {acc}")

    return nodes

def evaluate_nodes(nodes):
    print("\n📊 Final Evaluation Results:")
    for node in nodes:
        acc = node.evaluate()
        print(f"Node {node.id} Test Accuracy: {acc}")

# Run
nodes = secure_gossip_training_dh()
evaluate_nodes(nodes)

(1000, 30)
(334, 30)
(333, 30)
(333, 30)

🎯 Initial Training at Node 0
Node 0 encrypted model for Node 1 with shared key
Node 1 decrypted model
Node 1 trained model
Node 1 encrypted model for Node 2 with shared key
Node 2 decrypted model
Node 2 trained model
Node 2 encrypted model for Node 0 with shared key
Node 0 decrypted model
Node 0 trained model

📊 Accuracy after Round 1
Node 0 Test Accuracy: 0.7164179104477612
Node 1 Test Accuracy: 0.7164179104477612
Node 2 Test Accuracy: 0.7611940298507462
Node 0 encrypted model for Node 1 with shared key
Node 1 decrypted model
Node 1 trained model
Node 1 encrypted model for Node 2 with shared key
Node 2 decrypted model
Node 2 trained model
Node 2 encrypted model for Node 0 with shared key
Node 0 decrypted model
Node 0 trained model

📊 Accuracy after Round 2
Node 0 Test Accuracy: 0.7014925373134329
Node 1 Test Accuracy: 0.7611940298507462
Node 2 Test Accuracy: 0.7611940298507462
Node 0 encrypted model for Node 1 with shared key
Node 1 decrypted 